In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from swap.operations import create_swap_app, place_swap, cancel_swap, accept_swap, close_swap
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

staking_address = Account.from_mnemonic(os.environ.get("CREATOR_MN"))
team_wallet_address = Account.from_mnemonic(os.environ.get("TEAM_MN"))

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

Creator address: NBIIRJVCVD7ZWZGKCPLVZ5Z6QFUCXDE2GR4KQR5URM2ZBPV7V4VKPJFRZI
Bidder address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Seller Address: CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA


Alice is generating an example token...

In [4]:
asset1_id = int(os.environ.get("TOKEN_ID"))
asset2_id = int(os.environ.get("TOKEN2_ID"))
print(f"The token id is: {asset1_id}")
print(f"The token2 id is: {asset2_id}")

The token id is: 2989
The token2 id is: 3559


"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [5]:
deleteApps(client, [], creator)

In [6]:

appID = create_swap_app(
    client=client,
    creator=creator,
    staking_address=staking_address.get_address(),
    team_wallet_address=team_wallet_address.get_address()
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction XISRATLZSXM34464QTIA7JMHSWLP5PZIYNOTP34BOBGC4IGHR2VQ confirmed in round 218082.
Waiting for confirmation...
Transaction ZDMIRQO4JB5YDN3US744HSQXYFET3CWK4LIQWPP7TCG6EMZFDV7A confirmed in round 218084.
App ID: 4919
App Address: NRAWCBH3L5OFEHLZ7OPRWALYUYQBE2QPRMJZVWO2POP6A3ABVRNZCJ4FPM


Alice is setting up and funding token auction...

In [7]:
sellerAsset1BalanceBefore = get_balances(client, seller.get_address())[asset1_id]
print("Alice's asset1 balance: ", sellerAsset1BalanceBefore)

sellerAsset2BalanceBefore = get_balances(client, seller.get_address())[asset2_id]
print("Alice's asset2 balance: ", sellerAsset2BalanceBefore)

bidderAsset1BalanceBefore = get_balances(client, bidder.get_address())[asset1_id]
print("Carla's asset1 balance: ", bidderAsset1BalanceBefore)

bidderAsset2BalanceBefore = get_balances(client, bidder.get_address())[asset2_id]
print("Carla's asset2 balance: ", bidderAsset2BalanceBefore)

Alice's asset1 balance:  49999999857750
Alice's asset2 balance:  80000
Carla's asset1 balance:  50000000040050
Carla's asset2 balance:  49999999920000


In [8]:
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 100000}


In [9]:
offeringAmount = 10_000
acceptingAmount = 20_000


In [10]:

swap_index = place_swap(client=client, app_id=appID, offer=seller, offering_token_id=asset1_id, offering_token_amount=offeringAmount, accepting_token_id=asset2_id, accepting_token_amount=acceptingAmount, swap_index="")
print("Carla wants to swap on asset1 ", offeringAmount, " to asset2 ", acceptingAmount)
print("Carla bid", swap_index)

Waiting for confirmation...
Transaction IRM5OJKSUR2ELFAHFY4IL4KZNUKL7AKSUBRVHN3YLJX2FT7JYX2A confirmed in round 218086.
new address: UW5U6WICZGLYXQNZNHNGGM3VPYZDJW6ZE3X66UW52NNRW6EMD34GRDJ7FA
new private_key: tk6kgCXfWqQUiWkTqhS4Pd4KSjr/TCAkf5+hL9pEeVClu09ZAsmXi8G5adpjM3V+MjTb2Sbv71Ld01sbeIwe+A==
new passphrase: strategy muffin scare venture remember cinnamon emerge hat draw famous swing toilet between myth what erupt lottery wish soul wheel reform eye long able talent
Waiting for confirmation...
Transaction 4QC4WDZHGWINYMAUNBRKHD44M4SEWSYWJ2BWY47RCBWJWSRCR2LQ confirmed in round 218088.
Waiting for confirmation...
Transaction 2TWX4DU2WGSRA7UNUVTT62JBX6ODANN57JPVEDIOT646GLJDYSGQ confirmed in round 218090.
Waiting for confirmation...
Transaction XEJUAJ64WR52QHH4CFC6CFJGSKLLVVJBWANNUU7NFYYJGX7JMVHA confirmed in round 218092.
token_txn: {'sender': 'CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA', 'fee': 1000, 'first_valid_round': 218084, 'last_valid_round': 219084, 'note': None

In [11]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

cancel_swap(client, appID, seller, swap_index)
print("Carla cancel swap", swap_index)

Waiting for confirmation...
Transaction V26P4TCDSNU2YC7MK6K4BTQKIE3ZOU3IZZVPZACJN4JOBMMTOY3A confirmed in round 218096.
Carla cancel swap UW5U6WICZGLYXQNZNHNGGM3VPYZDJW6ZE3X66UW52NNRW6EMD34GRDJ7FA


In [12]:
swap_index = place_swap(client=client, app_id=appID, offer=seller, offering_token_id=asset1_id, offering_token_amount=offeringAmount, accepting_token_id=asset2_id, accepting_token_amount=acceptingAmount, swap_index="")
# replace swap
swap_index = place_swap(client=client, app_id=appID, offer=seller, offering_token_id=asset1_id, offering_token_amount=offeringAmount, accepting_token_id=asset2_id, accepting_token_amount=acceptingAmount, swap_index=swap_index)

new address: 5WC25QMAZ2JPRKPJJY5DMMVOXQAWO3CAN52CPLA3TXXAOYOQWUMEHRXS64
new private_key: ab9gVw936Ocd8R1q6DpcON4X1DB4/nz+qj/ieHnTRqHtha7BgM6S+KnpTjo2Mq68AWdsQG90J6wbne4HYdC1GA==
new passphrase: unfold load produce ill special ketchup material bulb ask two come toe cook crush screen wrist sort program cabin mixed very regular any absent appear
Waiting for confirmation...
Transaction 2JO6MG6IRS254BYBOCCF4A7LHMKVUXRXIBIWPCZ7YMLV7RXNFVUQ confirmed in round 218098.
Waiting for confirmation...
Transaction C3WNHBQF46PL5FCGIBPYQASXCZV3WIRJSWH4UBO4DLBS3UMYKYLA confirmed in round 218100.
Waiting for confirmation...
Transaction VZFMFYOT3WNGPJBWSROMIOCHSPSNVWINHKCOU4MAZV7YKRYCID7A confirmed in round 218102.
token_txn: {'sender': 'CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA', 'fee': 1000, 'first_valid_round': 218096, 'last_valid_round': 219096, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'S46yr60YjyJzK3Fk0J8V7mzMIOSxovaXAVqCHlwPl1A=', 'group': None, 'lease': None, 'type

In [13]:
accept_swap(client, appID, bidder, swap_index)

offer address CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA
offering_token_id 2989
offering_token_amount 10000
accepting_token_id 3559
accepting_token_amount 20000
Waiting for confirmation...
Transaction B2BFOVDRFHMFWZ4TEHRNMCVJ5RPJIUFVQ5RWXZ7SJWYRECXSRFRA confirmed in round 218108.


True

Alice is closing out the auction....

In [14]:
sleep(5)

close_swap(client, appID, creator, [asset1_id, asset2_id])

b'assets' [2989, 3559]
b'accounts' ['NBIIRJVCVD7ZWZGKCPLVZ5Z6QFUCXDE2GR4KQR5URM2ZBPV7V4VKPJFRZI', 'UCAX3M5HS7PWWLB6HDCM2O7JE4HC3RGBPNWFRP4SYU62ZS5NSS2GFOKIOU']
Waiting for confirmation...
Transaction 6Q4UNQHWZT2SCTA33JA42V3ZAV4YN34HNMFQ3XTAS7M34W72B64Q confirmed in round 218111.


In [15]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

The smart contract now holds the following: {0: 300000, 2989: 0, 3559: 0}


In [16]:
sellerAsset1BalanceAfter = get_balances(client, seller.get_address())[asset1_id]
print("Alice's asset1 balance: ", sellerAsset1BalanceAfter)

sellerAsset2BalanceAfter = get_balances(client, seller.get_address())[asset2_id]
print("Alice's asset2 balance: ", sellerAsset2BalanceAfter)

bidderAsset1BalanceAfter = get_balances(client, bidder.get_address())[asset1_id]
print("Carla's asset1 balance: ", bidderAsset1BalanceAfter)

bidderAsset2BalanceAfter = get_balances(client, bidder.get_address())[asset2_id]
print("Carla's asset2 balance: ", bidderAsset2BalanceAfter)

Alice's asset1 balance:  49999999847750
Alice's asset2 balance:  100000
Carla's asset1 balance:  50000000050050
Carla's asset2 balance:  49999999900000
